# SuppressImplicit
* Zero out any items the user has already seen

In [ ]:
function read_alpha_impl(alpha, split, implicit)
    @assert split in all_splits || split in all_raw_splits
    file = get_data_path("alphas/$(alpha)/predictions.jld2")
    if split in all_raw_splits
        df = JLD2.load(file, split)
    elseif implicit
        df = cat(JLD2.load(file, "explicit_" * split), JLD2.load(file, "implicit_" * split))
    else
        df = JLD2.load(file, "explicit_" * split)
    end
    df
end

function get_implicit_seen_probabilities(alpha)
    df = read_alpha_impl(alpha, "training", true)
    p_seen = zeros(Float32, num_users(), Threads.nthreads())
    Threads.@threads for i = 1:length(df.user)
        p_seen[df.user[i], Threads.threadid()] += df.rating[i]
    end
    vec(sum(p_seen, dims = 2))
end

@memoize function read_alpha(alpha, split, implicit)
    @assert !occursin("training", split)
    df = read_alpha_impl(alpha, split, implicit)
    if implicit
        # zero out any items the user has already seen
        p_seen = get_implicit_seen_probabilities(alpha)
        Threads.@threads for i = 1:length(df.rating)
            df.rating[i] /= 1 - p_seen[df.user[i]]
        end
    end
    df
end;